In [6]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
#get list of cards from netrunnerdb
f =  open('cards','w')
r =  requests.get('https://netrunnerdb.com/api/2.0/public/cards')
f.write(r.text)
f.close()

In [5]:
f =  open('decks','w')

#make list of dates
date1 = '2014-08-27'
date2 = '2020-06-07'
dates = pd.date_range(date1, date2).strftime('%Y-%m-%d')

#we need to make a new json object containing each dates decklists json object
f.write('{')

#Write out into file 
for day in dates:
    r =  requests.get('https://netrunnerdb.com/api/2.0/public/decklists/by_date/'+day)
    f.write('"'+day+'": ['+r.text)
    f.write('],')

#last one hardcoded so there is not comma at the end...
lastday='2020-06-08'
r =  requests.get('https://netrunnerdb.com/api/2.0/public/decklists/by_date/'+lastday)
f.write('"'+lastday+'": ['+r.text)
f.write(']')

#end the json object and close the file
f.write('}')

f.close()

In [13]:
#create a pickle of cards database
cards = pd.read_json (r'cards')
cards = pd.json_normalize(cards['data'].values)

#fix some of the card database
cards['code'] = 'cards.'+cards['code']
#Remove a card that does not appear in any decks from the dataframe of cards
cards = cards[cards['code'] != 'cards.00012']

In [15]:
#create a pickle of decks database
df =  pd.read_json(r'decks')
deck_list = pd.concat([pd.json_normalize(x)['data'] for x in df.values])
mask = deck_list.map(lambda d: len(d)) > 0
deck_list = deck_list[mask]
decks = pd.concat([pd.json_normalize(x) for x in deck_list.values],sort=True,ignore_index=True)

#drop the columns that I don't care about
decks = decks.drop(['date_update','description','name','tournament_badge','user_id','user_name'],axis=1)
decks = decks.fillna(0)

#turn all of the deck numbers in np.int8 to reduce file size
decks_float = decks.select_dtypes(include=['float'])
converted = decks_float.astype(np.int8)
decks[converted.columns] = converted

#make list of idenity cards
identity_cards = cards[cards['type_code'] == 'identity']
#make two new features keep track of the idenity cards and the faction
for ident in identity_cards['code']:
    decks.loc[decks[ident]==1,'identity_card']=ident
    decks.loc[decks[ident]==1,'faction']= cards[cards['code']==ident]['faction_code'].values[0]
#delete old columns that corresponded to identiy cards
decks = decks.drop(identity_cards['code'],axis=1)

#clean up the last few columns types
decks['id']=pd.to_numeric(decks['id'], downcast='integer')
#decks.loc[:,'mwl_code':'faction'] = decks.loc[:,'mwl_code':'faction'].astype('category')
decks['date_creation'] = pd.to_datetime(decks['date_creation'])

#now I want to remove decks that are not really decks...the non-identity ones to start off
decks = decks[decks['faction'] != 'neutral-runner']
decks = decks[decks['faction'] != 'neutral-corp']

#now remove the draft format decks
draft_cards = ['cards.00005','cards.00006','cards.00007','cards.00008','cards.00009','cards.00010','cards.00011','cards.000013']
decks = decks[~decks['identity_card'].isin(draft_cards)]

In [17]:
cards.to_pickle('cards_processed.pkl')
decks.to_pickle('decks_processed.pkl')